In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.model_selection import GridSearchCV
import json
from collections import Counter

# Model Losowego Lasu (Random Forest)

## Zestaw danych

In [3]:
try:
    dataset = fetch_ucirepo(id=519)
    features = dataset.data.features[["age", "serum_creatinine", "ejection_fraction"]]
    target = dataset.data.targets
except Exception as e:
    df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
    df.columns = [col.lower() if col == "DEATH_EVENT" else col for col in df.columns]
    features = df[["age", "serum_creatinine", "ejection_fraction"]]
    target = df["death_event"]


df = pd.concat([features, target], axis=1)

In [3]:
counts = target.value_counts()
total = len(target)

for label, count in counts.items():
    percent = (count / total) * 100
    print(f"Liczba {'zmarłych' if label == (1,) else 'żywych'}: {count} ({percent:.2f}%)")

Liczba żywych: 203 (67.89%)
Liczba zmarłych: 96 (32.11%)


## Wstępne przetwarzanie danych

Random Forest nie wymaga skalowania danych, więc przetwarzanie danych zostaje pominięte.

## Zastosowanie modelu

In [4]:
X = df[["age", "serum_creatinine", "ejection_fraction"]]
y = df["death_event"]

In [5]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Tworzenie modelu lasu losowego
    model = RandomForestClassifier()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

results_df

,iteration,accuracy,precision,recall,f1_score
0,0,0.766667,0.640000,0.571429,0.603774
1,1,0.800000,0.681818,0.576923,0.625000
2,2,0.811111,0.620690,0.750000,0.679245
3,3,0.777778,0.655172,0.655172,0.655172
4,4,0.711111,0.450000,0.375000,0.409091
...,...,...,...,...,...
95,95,0.744444,0.650000,0.448276,0.530612
96,96,0.833333,0.739130,0.653846,0.693878
97,97,0.722222,0.605263,0.696970,0.647887
98,98,0.788889,0.652174,0.576923,0.612245


#### Ewaluacja modelu

In [6]:
stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.63,0.88,0.76
precision,0.41,0.85,0.63
recall,0.38,0.81,0.57
f1_score,0.41,0.76,0.60


#### Optymalizacja modelu

- Wyszukiwanie najlepszych parametrów z GridSearch

Metoda GridSearchCV jest jedną z popularnych technik optymalizacji modeli regresji logistycznej, automatyzującą wyszukiwanie najlepszych hiperparametrów, takich jak siła i typ regularyzacji. Poprawia wydajność modelu poprzez użycie walidacji krzyżowej, zapewniając solidność i możliwość uogólnienia na nowe dane.

`Testowane parametry:`
- n_estimators (liczba drzew w lesie): domyślnie wynosi 100, ale zwiększenie tej wartości może poprawić wyniki (kosztem czasu obliczeń)
- max_depth (głębokość drzewa): ograniczenie głębokości drzewa, aby uniknąć przeuczenia
- max_features (liczba cech do podziału): ograniczenie liczby cech branych pod uwagę przy każdym podziale zmniejsza ryzyko przeuczenia
- min_samples_leaf (minimalna liczba próbek w węźle liścia) i min_samples_split (minimalna liczba próbek do podziału węzła): zwiększenie tych wartości sprawia, że model staje się mniej podatny na przeuczenie
- class_weight: parametr ten pozwala przypisać różne wagi klasom w celu zrównoważenia ich wpływu na model. Jest szczególnie przydatny, gdy mamy nierównomierny rozkład klas (np. jedna klasa występuje dużo częściej niż druga)

In [ ]:
results_list = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Tworzenie modelu lasu losowego
    model = RandomForestClassifier()

    param_grid = {
        "n_estimators": [50, 75, 100],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2"],
        "class_weight": [
            "balanced",
            "balanced_subsample",
        ],
    }

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        n_jobs=-1,
        scoring="recall",
    )

    grid_search.fit(X_train, y_train)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results_list.append({"best_params": grid_search.best_params_})

results_gs = pd.DataFrame(results_list)

In [13]:
best_params_list = [json.dumps(result["best_params"]) for result in results_list]

counter = Counter(best_params_list)

# Sortowanie wyników od najczęstszych do najmniej częstych
sorted_results = sorted(counter.items(), key=lambda x: x[1], reverse=True)

print("Najczęściej występujące najlepsze parametry:")
for params, count in sorted_results[:3]:
    print(f"Parametry: {params} - Wystąpienia: {count}")

Najczęściej występujące najlepsze parametry:
Parametry: {"class_weight": "balanced", "max_depth": null, "max_features": "sqrt", "min_samples_leaf": 4, "min_samples_split": 10, "n_estimators": 100} - Wystąpienia: 4
Parametry: {"class_weight": "balanced", "max_depth": null, "max_features": "log2", "min_samples_leaf": 4, "min_samples_split": 10, "n_estimators": 75} - Wystąpienia: 4
Parametry: {"class_weight": "balanced", "max_depth": 10, "max_features": "sqrt", "min_samples_leaf": 4, "min_samples_split": 10, "n_estimators": 50} - Wystąpienia: 4


In [5]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Tworzenie modelu lasu losowego
    model = RandomForestClassifier(class_weight="balanced", max_depth=10, max_features="sqrt", min_samples_leaf=4, min_samples_split=10, n_estimators=50)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.67,0.90,0.77
precision,0.44,0.81,0.61
recall,0.50,0.87,0.72
f1_score,0.50,0.82,0.66
